In [10]:
from math import *
from fractions import Fraction
import random

import numpy as np
import pandas as pd

from qiskit import QuantumCircuit
from qiskit.visualization import plot_histogram, plot_bloch_multivector
from qiskit.quantum_info import DensityMatrix
from qiskit.quantum_info import Statevector
from qiskit_braket_provider import BraketLocalBackend


class QuantumCircuitGenerator:

    hexColorArray = []
    gValColorArray = []
    thetaColorArray = []

    qubitCount = 0

    qCircuit = None
    encryptionCircuit = []
    encryptionKey = None

    def __init__(self, colorArray, encryptionKey) -> None:
        self.encryptionKey = encryptionKey
        self.hexColorArray = colorArray
        for hexColor in self.hexColorArray:
            hexColor = hexColor.lstrip("#")
            RGB = tuple(int(hexColor[i:i+2], 16) for i in (0, 2, 4))
            gVal = RGB[0]
            self.gValColorArray.append(gVal)
            theta = self.gVal2Theta(gVal)
            self.thetaColorArray.append(theta)
        self.qubitCount = len(self.hexColorArray)

    def generateQuantumCircuit(self):
        qCirc = QuantumCircuit(self.qubitCount)
        for pos in range(self.qubitCount):
            theta = self.gVal2Theta(self.thetaColorArray[pos])
            qCirc.ry(theta,pos)
        self.qCircuit = qCirc

    def gVal2Theta(self, gValue):
        return (gValue/255)*pi
    
    def applyEncryption(self):
        for index in range(len(self.encryptionKey)):
            match self.encryptionKey[index]:
                case "X":
                    self.qCircuit.x(index)
                case "Y":
                    self.qCircuit.y(index)
                case "Z":
                    self.qCircuit.z(index)
                case "H":
                    self.qCircuit.h(index)
                case "S":
                    self.qCircuit.s(index)
                case "C":
                    self.qCircuit.id(index)

#~~~~~
c = QuantumCircuitGenerator(["#000000" for x in range(25)], [random.choice(["X","Y","Z","H","S","C"]) for x in range(25)])
c.generateQuantumCircuit()
c.applyEncryption()
c.qCircuit.draw()

┌───────┐┌───┐
 q_0: ┤ Ry(0) ├┤ S ├
      ├───────┤├───┤
 q_1: ┤ Ry(0) ├┤ Y ├
      ├───────┤├───┤
 q_2: ┤ Ry(0) ├┤ S ├
      ├───────┤├───┤
 q_3: ┤ Ry(0) ├┤ S ├
      ├───────┤├───┤
 q_4: ┤ Ry(0) ├┤ I ├
      ├───────┤├───┤
 q_5: ┤ Ry(0) ├┤ I ├
      ├───────┤├───┤
 q_6: ┤ Ry(0) ├┤ Y ├
      ├───────┤├───┤
 q_7: ┤ Ry(0) ├┤ X ├
      ├───────┤├───┤
 q_8: ┤ Ry(0) ├┤ X ├
      ├───────┤├───┤
 q_9: ┤ Ry(0) ├┤ S ├
      ├───────┤├───┤
q_10: ┤ Ry(0) ├┤ Z ├
      ├───────┤├───┤
q_11: ┤ Ry(0) ├┤ Z ├
      ├───────┤├───┤
q_12: ┤ Ry(0) ├┤ X ├
      ├───────┤├───┤
q_13: ┤ Ry(0) ├┤ S ├
      ├───────┤├───┤
q_14: ┤ Ry(0) ├┤ Z ├
      ├───────┤├───┤
q_15: ┤ Ry(0) ├┤ Z ├
      ├───────┤├───┤
q_16: ┤ Ry(0) ├┤ H ├
      ├───────┤├───┤
q_17: ┤ Ry(0) ├┤ H ├
      ├───────┤├───┤
q_18: ┤ Ry(0) ├┤ I ├
      ├───────┤├───┤
q_19: ┤ Ry(0) ├┤ I ├
      ├───────┤├───┤
q_20: ┤ Ry(0) ├┤ X ├
      ├───────┤├───┤
q_21: ┤ Ry(0) ├┤ S ├
      ├───────┤├───┤
q_22: ┤ Ry(0) ├┤ Y ├
      ├───────┤├───┤
q_23: ┤ Ry(0) ├┤ Z ├
      ├───────┤├───┤
q_24: ┤ Ry(0) ├┤ H ├
      └───────┘└───┘